In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
!pip install transformers==4.9.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
import torch
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM
import pandas as pd

In [25]:
tokenizer = AutoTokenizer.from_pretrained("noahkim/KoT5_news_summarization")
model = AutoModelForSeq2SeqLM.from_pretrained("noahkim/KoT5_news_summarization")

In [7]:
import numpy as np
import random

def set_seed(random_seed):
    torch.random.manual_seed(random_seed)
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)


In [8]:
set_seed(777)

In [26]:
companyN = ['HJ중공업','SBI핀테크솔루션즈','화인베스틸']

In [ ]:
whole_company = pd.DataFrame(columns = {'title','label','content','score','company'})
no_info_company = []

for company in companyN:
  input_path = f'/content/drive/MyDrive/Project3/3.Classification/out/_label_title_{company}_2023.2.22_10시시.csv'
  input_f = pd.read_csv(input_path)

  if len(input_f) == 0:
    no_info_company.append(company)
  else:
    input_f['company'] = [company for i in range(len(input_f))]
    whole_company = pd.concat([whole_company,input_f])
    
whole_company

In [ ]:
news = whole_company['content']
summary = []

for index, content in enumerate(news):
    text = news[index]
    input_ids = tokenizer.encode(text)
    gen_ids = model.generate(torch.tensor([input_ids]),
                              max_length=1000,repetition_penalty=1.4,
                              top_k=6,
                              temperature=1.0,                          
                              pad_token_id=tokenizer.pad_token_id,
                              eos_token_id=tokenizer.eos_token_id,
                              bos_token_id=tokenizer.bos_token_id)
    generated = tokenizer.decode(gen_ids[0,:].tolist())
    summary.append(generated)
    
summary 

In [93]:
import re

summary = [re.sub("<pad>|</s>", "", summ) for summ in summary]

In [94]:
output = whole_company
output['summary'] = summary

# neutral인 기사는 제거
output = output[output['label'] != 'neutral']

# 정보 없는 회사는 따로 처리
for company in no_info_company:
  empty_df = pd.DataFrame(columns = {'title','label','content','score','company','summary'})
  empty_df['company'] = [company]

  output = pd.concat([output,empty_df])

output = output.astype(object).replace(np.nan, 'None')

output.loc[output['summary'] == 'None','summary'] = '기사 없음'
output


,label,title,score,content,company,summary
2,positive,"HJ중공업, 한국토지신탁·미래에셋증권과 선박금융 STO 활성화 협약",0.688783,블록체인 기술 활용 토큰 발행 사업 발굴선박 권리 ‘토큰화’…분할소유 개념 도입[이...,HJ중공업,"HJ중공업은 한국토지신탁, 미래에셋증권과 함께 선박금융 관련 증권형 토큰(STO) ..."
3,positive,"HJ중공업, 10년 만에 3167억 규모 중대형 상선 수주",0.999877,HMM과 9000TEU급 건조 계약메탄올 추진 컨테이너선 2척친환경 선박 기술력 재...,HJ중공업,"부산 조선업계가 10년만에 중대형상선을 수주했는데, 이는 수년째 정부의 조선산업 지..."
4,positive,"HJ중공업, 메탄올 추진선 수주 첫 성공",0.999882,HMM과 9000TEU급 메탄올 추진 컨선 2척 계약“그동안 축적한 친환경선 건조 ...,HJ중공업,HJ중공업은 국제해사기구의 2050년 해상 탄소중립 정책과 이에 따른 탈(脫) 탄소...
5,positive,HJ중공업 9000TEU 컨테이너선 수주…기자재 업계도 일감 확대 기대,0.999863,HJ중공업 조선부문이 친환경 선박인 9000TEU급 메탄올 추진선 건조를 수주하면서...,HJ중공업,"HJ중공업은 15일, 차세대 친환경 선박인 9000TEU급 메탄올 추진선을 건조하면..."
6,positive,"HJ중공업, 9000TEU급 메탄올 추진 컨테이너선 2척 수주",0.99986,HJ중공업이 국적 해운사 HMM(옛 현대상선)으로부터 메탄올 추진 컨테이너선박 2척...,HJ중공업,HJ중공업은 국제해사기구의 2050년 해상 탄소중립 정책과 이에 따른 탈탄소 선박 ...
7,positive,"HJ중공업, 9000TEU급 메탄올 추진 컨테이너선 2척 수주",0.99986,기사내용 요약친환경 메탄올 추진선 첫 수주[부산=뉴시스] 하경민 기자 = HJ중공업...,HJ중공업,HJ중공업 조선부문은 국제해사기구(IMO)의 2050년 해상 탄소중립 정책과 이에 ...
0,None,None,None,None,SBI핀테크솔루션즈,기사 없음
0,None,None,None,None,화인베스틸,기사 없음


In [95]:
output.to_csv('/content/drive/MyDrive/Project3/4.Summarization/out/_summ_2023.2.22_10시.csv')